In [1]:
!nvidia-smi

Sat Feb 17 22:54:30 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.112      Driver Version: 528.92       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| N/A   36C    P8    16W / 120W |    689MiB /  8192MiB |     14%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import os, shutil
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.docstore.document import Document
from langchain.text_splitter import NLTKTextSplitter, CharacterTextSplitter, RecursiveCharacterTextSplitter


doc_dir = "data/text_chunks"

n = 2500
m = 500

preprocessor = RecursiveCharacterTextSplitter(
    chunk_size=n,
    chunk_overlap=m,
    # separators=[" "],
    keep_separator=False,
    add_start_index=False,
    strip_whitespace=False
)

file_path = './sample_pdf.pdf'
loader = PyPDFLoader(file_path)
pages = loader.load()
# converting text to langchain document format
converted_doc = [Document("".join([page.page_content for page in pages]))]
# splitting document into chunks
docs = preprocessor.split_documents(converted_doc)


## OPTIONAL PART ##
try:
  shutil.rmtree(doc_dir)
  print("creating data folder again..")
except:
  print("creating data folder..")

os.makedirs(doc_dir)

chunks = [k.page_content for k in docs]
for chunk_id in range(len(chunks)):
  text_file_path = f'{doc_dir}/{chunk_id}.txt'
  with open(text_file_path, 'w') as f:
      f.write(chunks[chunk_id])

creating data folder again..


In [6]:
len(docs)

5

In [7]:
len(docs[1].page_content), len(docs[1].page_content.split())

(2434, 385)

In [8]:
####
# initializing document store

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma, FAISS


# embeddings = OpenAIEmbeddings(
#     model="text-embedding-ada-002",
# )
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# db = Chroma.from_documents(docs, embeddings)
db = FAISS.from_documents(docs, embeddings)

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":3})

/home/apurva/anaconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from langchain.llms import OpenAI, OpenAIChat
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

MODEL_NAME = "TheBloke/Llama-2-7b-Chat-GPTQ"
 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
 
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

CUDA extension not installed.


In [10]:
model.device

device(type='cuda', index=0)

In [11]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15
 
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)
 
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0.1})

In [12]:
result = llm.invoke(
    "Explain the difference between ChatGPT and open source LLMs in a couple of lines."
)
print(result)


ChatGPT is an AI-powered chatbot developed by Meta AI that can understand and respond to user input in a conversational manner. Open source LLMs, on the other hand, are language models that are available for anyone to use and modify, with some examples including BERT, RoBERTa, and XLNet. While both types of models have their own strengths and weaknesses, they differ in terms of their architecture, training data, and licensing restrictions.


In [10]:
from langchain.prompts import PromptTemplate

# Build prompt
prompt_template = """Follow the following set of instructions that describes a task:
1. Use the following pieces of context to extract an answer to the question at the end.
2. Do not add any explaination/justification for answers and do not attempt to asnwer more than than what is asked.
3. Do not add anything out of the given context.

Context: {context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [11]:
print ("------- Prompt Begin -------")

final_prompt = PROMPT.format(context='CONTEXT', question="What is Global Warming?")
print(final_prompt)

print ("------- Prompt End -------")

------- Prompt Begin -------
Follow the following set of instructions that describes a task:
1. Use the following pieces of context to extract an answer to the question at the end.
2. Do not add any explaination/justification for answers and do not attempt to asnwer more than than what is asked.
3. Do not add anything out of the given context.

Context: CONTEXT

Question: What is Global Warming?
Answer:
------- Prompt End -------


In [12]:
# llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.1, max_tokens=-1)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False,
    chain_type_kwargs={"prompt": PROMPT, "verbose": True})

query = "What are the causes of Global Warming?"
result = qa({"query": query})
result

/home/apurva/anaconda3/envs/nlp/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Follow the following set of instructions that describes a task:
1. Use the following pieces of context to extract an answer to the question at the end.
2. Do not add any explaination/justification for answers and do not attempt to asnwer more than than what is asked.
3. Do not add anything out of the given context.

Context: also brings climate change and pollution with it. Industrialization is a major cause of the emission of 
not just carbon -containing gases but also poisonous gases like monoxide which ca n cause cancer, it 
is also a significant reason for overpopulation and deforestation since it needs thousands of acres 
of land to set up the industry.  
Waste Disposal  
Improper disposal of solid waste directly contributes to the greenhouse effect since it produces high 
amounts of methane and nitrous oxides which eventually increases the temperature of the planet.   
Effect

{'query': 'What the causes of Global Warming?',
 'result': ' Based on the provided text, the main causes of global warming are:\nEmission of greenhouse gases from smoke stacks, cars, and burning forests.\nDeforestation.\nBurning of fossil fuels.'}

In [13]:
from langchain.prompts import PromptTemplate

# Build prompt
prompt_template = """Follow the following set of instructions that describes a task:
1. Use the context and the chat history provided below to answer to the question at the end.
2. Do not add any explaination/justification for answers and do not attempt to asnwer more than than what is asked.
3. Do not add anything out of the given context.

Context:
{context}

Question: {question}
Answer:"""
CHAT_PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [14]:
print ("------- Prompt Begin -------")

final_prompt = CHAT_PROMPT.format(context='CONTEXT', chat_history="HISTORY", question="What is Global Warming?")
print(final_prompt)

print ("------- Prompt End -------")

------- Prompt Begin -------
Follow the following set of instructions that describes a task:
1. Use the context and the chat history provided below to answer to the question at the end.
2. Do not add any explaination/justification for answers and do not attempt to asnwer more than than what is asked.
3. Do not add anything out of the given context.

Context:
CONTEXT

Question: What is Global Warming?
Answer:
------- Prompt End -------


In [17]:
from langchain.chains import ConversationalRetrievalChain, LLMChain, StuffDocumentsChain
from langchain.memory import ConversationBufferWindowMemory
from src.prompt_templates import (
    # the QA prompt
    DEFAULT_PROMPT_TEMPLATE,
    # the condense question prompt
    DEFAULT_CHAT_HISTORY_PROMPT,
)

memory_chain = ConversationBufferWindowMemory(
    memory_key="chat_history", input_key="question", output_key="answer", return_messages=True, k=2
)

# qa = ConversationalRetrievalChain.from_llm(
#     llm=llm,
#     retriever=retriever,
#     memory=memory_chain,
#     return_source_documents=True,
#     # get_chat_history=_get_chat_history,
#     # condense_question_prompt=CHAT_PROMPT,
#     combine_docs_chain_kwargs = {"prompt": CHAT_PROMPT},
#     chain_type="stuff",  # 'refine',
#     output_key='answer',
#     verbose=True
# )
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory_chain,
    return_source_documents=True,
    # get_chat_history=_get_chat_history,
    condense_question_llm = llm,
    condense_question_prompt=DEFAULT_CHAT_HISTORY_PROMPT,
    combine_docs_chain_kwargs = {"prompt": DEFAULT_PROMPT_TEMPLATE},    # the LLMChain prompt to get the answer
    chain_type="stuff",  # 'refine',
    output_key='answer',
    verbose=True
)

In [18]:
qa("What are the effects of global warming?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Follow the following set of instructions that describes a task:
1. Use the context provided below to answer to the question at the end.
2. Do not add any explaination/justification for answers and do not attempt to asnwer more than than what is asked.
3. Do not add anything out of the given context.

Context:
GLOBAL WARMING  
Our planet Earth is home to innumerable species of both flora and fauna, but out of all, it is home to 
one of the most intelligent and devastating species which is us, humans. As much as we are 
important to utilise and protect the weak in the world, we are also causing irreparable damage to the 
world. Many climate scientists believe that without significant reductions in greenhouse gas 
emissions, the 21st century will see temperatures rise by 3 to 8 degrees, climatic patterns piercingly 
alter, ice sheets collapse, and seas rise several feet.   
With the p

{'question': 'What are the effects of global warming?',
 'chat_history': [],
 'answer': ' Global warming leads to severe weather conditions including heavy rainfall, snowfall, and storms.',
 'source_documents': [Document(page_content="GLOBAL WARMING  \nOur planet Earth is home to innumerable species of both flora and fauna, but out of all, it is home to \none of the most intelligent and devastating species which is us, humans. As much as we are \nimportant to utilise and protect the weak in the world, we are also causing irreparable damage to the \nworld. Many climate scientists believe that without significant reductions in greenhouse gas \nemissions, the 21st century will see temperatures rise by 3 to 8 degrees, climatic patterns piercingly \nalter, ice sheets collapse, and seas rise several feet.   \nWith the possible exception of another world war, a massive asteroid, a deadly pandemic, or global \nwarming may be the only threats to our planet. The release of greenhouse gases such 

In [1]:
import logging
from src.chat_engine import ChatBotModel

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

chat_bot = ChatBotModel(logger, model_name="TheBloke/Llama-2-7b-Chat-GPTQ", embedding_model_name="all-MiniLM-L6-v2")

/home/apurva/anaconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TheBloke/Llama-2-7b-Chat-GPTQ


CUDA extension not installed.


all-MiniLM-L6-v2


In [2]:
qa = chat_bot.create_qa_instance(file_path="./sample_pdf.pdf")

5


In [4]:
qa("Can you elaborate your answer?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What are the effects of global warming?
Assistant:  Global warming leads to severe weather conditions including heavy rainfall, snowfall, and storms.

Follow Up Input: Can you elaborate your answer?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Follow the following set of instructions that describes a task:
1. Use the context provided below to answer to the question at the end.
2. Do not add any explaination/justification for answers and do not attempt to asnwer more than than what is asked.
3. Do not add anything out of the given context.

Context:
GLOBAL WARMING  
Our planet Earth is home to innumerable species of both flora and fauna, but out of all, it is hom

{'question': 'Can you elaborate your answer?',
 'chat_history': [HumanMessage(content='What are the effects of global warming?'),
  AIMessage(content=' Global warming leads to severe weather conditions including heavy rainfall, snowfall, and storms.')],
 'answer': "  Due to global warming, there is a rise in the average temperature of the earth's surface, especially one that is long enough to produce climate change, is referred to as global warming. Since 1900, the global average face temperature has risen by more than a degree, and the rate of warming has been nearly three times the century-long average since 1970. This rise in temperature leads to severe weather conditions such as intense heat waves, heavy rainfall, and frequent wildfires.",
 'source_documents': [Document(page_content="GLOBAL WARMING  \nOur planet Earth is home to innumerable species of both flora and fauna, but out of all, it is home to \none of the most intelligent and devastating species which is us, humans. As mu